In [5]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots

import sys, os
sys.path.insert(0, os.path.join(os.getcwd(), '..', 'scripts'))


In [6]:
from etl_pipeline import ejecutar_etl

df_limpio = ejecutar_etl(
    ruta_entrada='../data/csv_raw',
    ruta_salida='../data/csv_limpio/mallas_curriculares_limpio.csv'
)
df_limpio.head()

ETL Pipeline — Mallas Curriculares

[1/7] Cargando CSV raw...
  → 500 filas cargadas de 12 archivos

[2/7] Reparando filas corruptas...
  → 5 filas corruptas reparadas en tecnico_electronica_compu.csv

[3/7] Limpiando whitespace y \xa0...

[4/7] Normalizando etiquetas Bloom...
  → 11 etiquetas únicas → 6 etiquetas únicas
  → Valores: ['Analizar', 'Aplicar', 'Comprender', 'Crear', 'Evaluar', 'Recordar']

[5/7] Normalizando area_dominio...
  → 22 áreas únicas → 19 áreas únicas
  → Valores: ['AGRONOMIA', 'ENFERMERIA', 'INGENIERIA_INFORMATICA', 'LIC_CIENCIAS_JURIDICAS', 'LIC_ECONOMIA', 'LIC_ESTADISTICA_CIENCIA_DATOS', 'LIC_IDIOMAS', 'LIC_MATEMATICA', 'LIC_SOCIOLOGIA', 'LIC_VETERINARIA_ZOOTECNIA', 'MEDICINA_GENERAL', 'NUTRICION', 'ODONTOLOGIA', 'PROFESORADO_LENGUAJE_LITERATURA', 'TECNICO_EN_GESTION_TALENTO_HUMANO', 'TECNICO_EN_MERCADEO_VENTAS', 'TECNICO_EN_RELACIONES_PUBLICAS', 'TECNICO_INGENIERIA_COMPUTACION', 'TECNICO_INGENIERIA_ELECTRONICA']

[6/7] Preparando texto para TF-IDF (stemming 

,id,area_dominio,nombre_asignatura,resultado_aprendizaje,nivel_bloom,competencia_laboral_relacionada,complejidad_estructural,archivo_origen,texto_limpio_tfidf,texto_transformer
0,1,AGRONOMIA,Ciencia de Suelos,Identificar las propiedades físicas químicas y...,Comprender,Diseño y Manejo de Sistemas de Riego Eficiente,2,malla_agronomia.csv,identific propiedad fisic quimic biolog suel c...,Identificar las propiedades físicas químicas y...
1,2,AGRONOMIA,Fitopatología General,Reconocer los principales grupos de patógenos ...,Recordar,Gestión Integral de Suelos Agrícolas,2,malla_agronomia.csv,reconoc principal grup patogen vegetal clasifi...,Reconocer los principales grupos de patógenos ...
2,3,AGRONOMIA,Agroclimatología,Comprender la interacción entre factores climá...,Comprender,Implementación de Agricultura de Precisión,3,malla_agronomia.csv,comprend interaccion factor climat desarroll v...,Comprender la interacción entre factores climá...
3,4,AGRONOMIA,Botánica Agrícola,Identificar estructuras morfológicas y anatómi...,Recordar,Manejo Integrado de Plagas y Enfermedades,2,malla_agronomia.csv,identific estructur morfolog anatom plant cult...,Identificar estructuras morfológicas y anatómi...
4,5,AGRONOMIA,Manejo de Fertilidad de Suelos,Aplicar técnicas de muestreo y análisis de sue...,Aplicar,Producción y Certificación de Semillas de Calidad,4,malla_agronomia.csv,aplic tecnic muestre analisis suel formul plan...,Aplicar técnicas de muestreo y análisis de sue...


## Modelo baseline: TF-IDF + RandomForest con datos limpios

Mejoras respecto al modelo anterior:
- **Bloom normalizado**: 6 etiquetas (antes 11 con duplicados sustantivo/verbo)
- **Stop words en español** (antes usaba inglés)
- **Texto lematizado** con stemming SnowballStemmer español
- **Filas corruptas reparadas**: 7 filas con comas sin escapar

In [7]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report, f1_score
from sklearn.pipeline import Pipeline

In [8]:
df = pd.read_csv('../data/csv_limpio/mallas_curriculares_limpio.csv')

X = df['texto_limpio_tfidf']
y = df['nivel_bloom']

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

pipeline = Pipeline([
    ('tfidf', TfidfVectorizer(max_features=10000)),
    ('classifier', RandomForestClassifier(n_estimators=100, random_state=42))
])

pipeline.fit(X_train, y_train)
y_pred = pipeline.predict(X_test)

f1_per_class = f1_score(y_test, y_pred, average=None)
class_names = pipeline.named_steps['classifier'].classes_

print("Precisión:", accuracy_score(y_test, y_pred))
print("F1-score por clase:")
for class_name, f1 in zip(class_names, f1_per_class):
    print(f"  {class_name}: {f1:.4f}")

print(f"\nF1-score promedio (macro): {f1_score(y_test, y_pred, average='macro'):.4f}")
print("\nReporte de clasificación:")
print(classification_report(y_test, y_pred))

Precisión: 0.93
F1-score por clase:
  Analizar: 0.9730
  Aplicar: 0.9020
  Comprender: 0.8235
  Crear: 0.9231
  Evaluar: 0.9744
  Recordar: 0.9412

F1-score promedio (macro): 0.9228

Reporte de clasificación:
              precision    recall  f1-score   support

    Analizar       0.95      1.00      0.97        18
     Aplicar       0.92      0.88      0.90        26
  Comprender       0.88      0.78      0.82         9
       Crear       0.90      0.95      0.92        19
     Evaluar       0.95      1.00      0.97        19
    Recordar       1.00      0.89      0.94         9

    accuracy                           0.93       100
   macro avg       0.93      0.92      0.92       100
weighted avg       0.93      0.93      0.93       100

